GAT
- currently able to run on homogeneous cora datasets
- need to run this on heterogeneous AMiner, DBLP

In [1]:
# choose which dataset you want to run on. Default: AMiner
user_dataset = "Planetoid"
# For heterogeneous: choose which label you want to compute loss against. Default: Author
label = "author"

In [2]:
!pip install torch_geometric > /dev/null

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GATConv
import torch.nn.functional as F

In [4]:
### arguments.py File ###
### Modified for colab since colab does not seem to like argparse ###

# Defined custom class to hold arguments
class Args:
  def __init__(self):
    self.root_dir = "/content"
    self.data_dir = "/content/data"
    self.epochs = 300
    self.runs = 5
    self.droput = 0.4
    self.lr = 0.001
    self.wd = 0.001
    self.num_layers = 2
    self.num_hidden = 8
    self.num_features = 0 # placeholder
    self.num_classes = 0 # placeholder

def add_data_features(args, data):
  args.num_features = data.x.shape[1]
  args.num_classes = data.y.shape[0]
  return args

In [5]:
### data.py File ###

args = Args()

if user_dataset == "AMiner":
  from torch_geometric.datasets import AMiner
  dataset = AMiner(root=args.root_dir)
elif user_dataset == "DBLP":
  from torch_geometric.datasets import DBLP
  dataset = DBLP(root=args.root_dir)
elif user_dataset == "Planetoid":
  from torch_geometric.datasets import Planetoid
  dataset = Planetoid(root=args.root_dir, name="Cora")
else:
  print(f"ERROR! Your dataset {user_dataset} was not found.")
  exit()


print(dataset[0])

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


Processing...
Done!


In [13]:
### model.py File ###

def make_layers(self):
    layers = []
    # initialize layers in a loop that uses conditionals to determine the input and output dimensions of the feature vectors
    for i in range(self.num_layers):
        if i == 0:  # first layer
            # dimensions in = input data size
            # dimensions out = hidden layer size
            layer = GATConv(dataset.num_features, self.num_hidden, heads=self.in_head, dropout=0.6)

        elif i < self.num_layers - 1: # hidden layer(s)
            # dimensions in = hidden layer size
            # dimensions out = hidden layer size
            # shouldn't hit this case for only two layers anyways
            layer = GATConv(self.num_hidden, self.num_hidden)


        else:  # output layer
            # dimensions in = hidden layer size
            # dimensions out = output size
            layer = GATConv(self.num_hidden*self.in_head, self.num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)

        layers.append(layer)

    return nn.ModuleList(layers)

class GAT_model(torch.nn.Module):
    def __init__(self, args):
        #super(GAT_model, self).__init__()
        super().__init__()
        self.num_hidden = args.num_hidden
        self.in_head = 8
        self.out_head = 1
        self.num_features = dataset.num_features
        self.num_layers = args.num_layers
        self.num_classes = dataset.num_classes
        self.layers = make_layers(self)
        self.lr = args.lr

    def forward(self, x, edge_idx):
      for i, layer in enumerate(self.layers):

        # apply the Attention layer
        x = F.dropout(x, p=0.6, training=self.training)
        x = layer(x, edge_idx)

        # Since I did not apply the activation function in the Layers array, I apply it using conditionals (to decide relu or softmax) here
        if i != len(self.layers) - 1:
          x = F.elu(x)
        else:
          x = F.log_softmax(x, dim = 1)

      return x



In [14]:
### main.py File ###

def train(model, X, Y, data):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr = model.lr)
    optimizer.zero_grad()
    activations = model(X, data.edge_index)

    # only calculate loss on train labels!!
    loss = F.nll_loss(activations[data.train_mask], Y[data.train_mask])
    loss.backward()
    optimizer.step()

def get_masked_acc(activations, y_true, mask):
    length = activations[mask].shape[0]
    correct = 0
    for yhat, y in zip(activations[mask], y_true[mask]):
        if torch.argmax(yhat) == y:
            correct += 1

    return correct / length

def get_accuracy(activations, y_true, data):
    train_acc = get_masked_acc(activations, y_true, data.train_mask)
    test_acc = get_masked_acc(activations, y_true, data.test_mask)
    val_acc = get_masked_acc(activations, y_true, data.val_mask)
    return train_acc, test_acc, val_acc

def main():
    # use gpu if possible (works most of the time here on colab)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Device: {device}")

    # get data
    data = dataset[0].to(device)
    x = data.x
    y = data.y

    # get preferences
    args = Args()
    args = add_data_features(args, data)


    for run in range(args.runs):
        # initialize model
        model = GAT_model(args).to(device)
        print("\n------------ new model ------------\n")
        for epoch in range(args.epochs):
          # log loss every 50 steps
            if epoch % 50 == 0 or epoch == args.epochs - 1:
                model.eval()
                activations = model(x, data.edge_index)
                loss = F.nll_loss(activations, y)
                train_acc, test_acc, val_acc = get_accuracy(activations, y, data)
                print(f" Epoch: {epoch} | Total Loss: {loss} | Train Accuracy: {train_acc} | Test Accuracy: {test_acc} | Val Accuracy: {val_acc}")

            # backprop & update
            train(model, x, y, data)

if __name__ == '__main__':
    main()

Device: cuda

------------ new model ------------

 Epoch: 0 | Total Loss: 1.9295308589935303 | Train Accuracy: 0.1357142857142857 | Test Accuracy: 0.216 | Val Accuracy: 0.236
 Epoch: 50 | Total Loss: 1.3065060377120972 | Train Accuracy: 0.9428571428571428 | Test Accuracy: 0.774 | Val Accuracy: 0.782
 Epoch: 100 | Total Loss: 0.8845387101173401 | Train Accuracy: 0.9642857142857143 | Test Accuracy: 0.802 | Val Accuracy: 0.794
 Epoch: 150 | Total Loss: 0.6898713707923889 | Train Accuracy: 0.9785714285714285 | Test Accuracy: 0.81 | Val Accuracy: 0.8
 Epoch: 200 | Total Loss: 0.6129773259162903 | Train Accuracy: 0.9928571428571429 | Test Accuracy: 0.81 | Val Accuracy: 0.796
 Epoch: 250 | Total Loss: 0.5817471146583557 | Train Accuracy: 0.9928571428571429 | Test Accuracy: 0.813 | Val Accuracy: 0.794
 Epoch: 299 | Total Loss: 0.5772804021835327 | Train Accuracy: 0.9928571428571429 | Test Accuracy: 0.811 | Val Accuracy: 0.792

------------ new model ------------

 Epoch: 0 | Total Loss: 1.937